>### 🚩 *Create a free WhyLabs account to get more value out of whylogs!*<br> 
>*Did you know you can store, visualize, and monitor whylogs profiles with the [WhyLabs Observability Platform](https://whylabs.ai/whylogs-free-signup?utm_source=whylogs-Github&utm_medium=whylogs-example&utm_campaign=Audio)? Sign up for a [free WhyLabs account](https://whylabs.ai/whylogs-free-signup?utm_source=whylogs-Github&utm_medium=whylogs-example&utm_campaign=Audio) to leverage the power of whylogs and WhyLabs together!*

# Extracting and Monitoring Audio Samples

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/whylabs/whylogs/blob/mainline/python/examples/experimental/whylogs_Audio_examples.ipynb)

This notebook demonstrates how to extract features from audio samples for the purpose of monitoring for drift/quality. 

*   The audio sample data is taken from Kaggle
*   The feature extraction is done using pyAudioAnalysis (https://github.com/tyiannak/pyAudioAnalysis/wiki/3.-Feature-Extraction) 
*   The logging of features is done by whylogs (https://github.com/whylabs/whylogs) 

For a full list of features that we extract and in-depth explanation of the pyAudioAnalysis set up, please refer to this blog post by the pyAudioAnalysis author: https://medium.com/behavioral-signals-ai/intro-to-audio-analysis-recognizing-sounds-using-machine-learning-20fd646a0ec5

In [2]:
# Note: you may need to restart the kernel to use updated packages.
%pip install 'whylogs[whylabs]' pyAudioAnalysis eyed3 pydub


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


**We will use Kaggle to get audio samples from a public Audio MNIST dataset.**

To set up Kaggle in Colab: https://www.kaggle.com/general/74235 

In [ ]:
! pip install -q kaggle
from google.colab import files
files.upload()

In [ ]:
! kaggle datasets download -d joserzapata/free-spoken-digit-dataset-fsdd

 32% 5.00M/15.5M [00:00<00:00, 34.9MB/s]
100% 15.5M/15.5M [00:00<00:00, 67.7MB/s]


In [ ]:
!unzip -q /content/free-spoken-digit-dataset-fsdd.zip -d .

replace ./.gitignore? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
# This cell is optional and specific to the Kaggle dataset we are using
# The dataset contains audio samples of the format [digit]_speaker_sampleNum.wav
# Example: 0_lucas_21.wav is the speaker Lucas saying digit ZERO, 21st sample 
# There are 500 samples total per speaker (50 for each digit between 0 and 9)
# We will organize the data by speaker for demo purposes, all samples
# from a specific speaker will be in a dedicated directory 
import os
import shutil

speakers = ["theo", "george", "jackson", "lucas", "nicolas", "yweweler"]
for speaker in speakers:
  path = os.path.join("/content/recordings", speaker) 
  os.mkdir(path) 

for filename in os.listdir("/content/recordings"):
  parts = filename.split("_")
  if(len(parts) > 1):
    shutil.move(os.path.join("/content/recordings/", filename), "/content/recordings/"+parts[1])


In [ ]:
# Set up the WhyLabs environment 
# The model type can be set as "Other"
# Documentation: https://docs.whylabs.ai/docs/whylabs-onboarding 

import getpass
import os

# set your org-id here - should be something like "org-xxxx"
print("Enter your WhyLabs Org ID") 
os.environ["WHYLABS_DEFAULT_ORG_ID"] = input()

# set your datased_id (or model_id) here - should be something like "model-xxxx"
print("Enter your WhyLabs Dataset ID")
os.environ["WHYLABS_DEFAULT_DATASET_ID"] = input()


# set your API key here
print("Enter your WhyLabs API key")
os.environ["WHYLABS_API_KEY"] = getpass.getpass()
print("Using API Key ID: ", os.environ["WHYLABS_API_KEY"][0:10])

In [ ]:
# Set up the audio feature extraction step with pyAudioAnalysis 
# https://medium.com/behavioral-signals-ai/intro-to-audio-analysis-recognizing-sounds-using-machine-learning-20fd646a0ec5

from pyAudioAnalysis import MidTermFeatures as aF
import os
import numpy as np
import plotly.graph_objs as go 
import plotly
import pandas as pd

# This example will focus on just one speaker, but can be set up with
# multiple directories 
dirs = ["/content/recordings/george"] 

# The feature extraction has the following parameters: 
# the mid-term window size (mw)
# the mid-term step size (ms)
# the short-term window size (sw)
# and the short-term step size (ss)
# These windows can be configured depending on the side of each audio sample
m_win, m_step, s_win, s_step = 0.5, 0.5, 0.1, 0.05 
# segment-level feature extraction:
features = [] 
for d in dirs: # get feature matrix for each directory 
    f, files, fn = aF.directory_feature_extraction(d, m_win, m_step, 
                                                   s_win, s_step) 
    features.append(f)
# (each element of the features list contains a 
# (samples x segment features) = (N x 138) feature matrix)

# Create a dataframe from the resulting features 
df = pd.DataFrame(f, columns = fn)


In [ ]:
# Now that we have all features extracted, we have a dataframe where features
# are columns and samples are rows. 
# We will use whylogs to profile the features dataframe to extract key statistics
# that we will tracking using the WhyLabs monitoring platform
# whylogs documentation: https://github.com/whylabs/whylogs 

import whylogs as why
from datetime import datetime, timedelta, timezone

# We are setting the date of profiling for easier navigation in the 
# WhyLabs platform 
current_date = datetime.now(timezone.utc)
profile = why.log(df).profile()
profile.set_dataset_timestamp(current_date)

In [ ]:
# As a final step, we write the resulting feature profile to the WhyLabs platform
# To see the result, log into https://hub.whylabsapp.com/ and navigate to the
# model you specified in the "Set up the WhyLabs environment" step 
from whylogs.api.writer.whylabs import WhyLabsWriter

# Write the log to the WhyLabs platform 
writer = WhyLabsWriter()
writer.write(file=profile.view())

(True, 'OK')

In [ ]:
# Optional: Extract features from samples generated by another speaker
# Submit profiles to the WhyLabs platform to compare feature differences
# between two speakers 
#https://medium.com/behavioral-signals-ai/intro-to-audio-analysis-recognizing-sounds-using-machine-learning-20fd646a0ec5

dirs = ["/content/recordings/lucas"] 

m_win, m_step, s_win, s_step = 0.5, 0.5, 0.1, 0.05 
# segment-level feature extraction:
features = [] 
for d in dirs: # get feature matrix for each directory
    f, files, fn = aF.directory_feature_extraction(d, m_win, m_step, 
                                                   s_win, s_step) 
    features.append(f)
# (each element of the features list contains a 
# (samples x segment features) = (10 x 138) feature matrix)

df_lucas = pd.DataFrame(f, columns = fn)
yesterday_date = datetime.now() - timedelta(days=1)
profile = why.log(df_lucas).profile()
profile.set_dataset_timestamp(yesterday_date)
writer = WhyLabsWriter()
writer.write(file=profile.view())


Analyzing file 1 of 500: /content/recordings/lucas/0_lucas_0.wav
Analyzing file 2 of 500: /content/recordings/lucas/0_lucas_1.wav
Analyzing file 3 of 500: /content/recordings/lucas/0_lucas_10.wav
Analyzing file 4 of 500: /content/recordings/lucas/0_lucas_11.wav
Analyzing file 5 of 500: /content/recordings/lucas/0_lucas_12.wav
Analyzing file 6 of 500: /content/recordings/lucas/0_lucas_13.wav
Analyzing file 7 of 500: /content/recordings/lucas/0_lucas_14.wav
Analyzing file 8 of 500: /content/recordings/lucas/0_lucas_15.wav
Analyzing file 9 of 500: /content/recordings/lucas/0_lucas_16.wav
Analyzing file 10 of 500: /content/recordings/lucas/0_lucas_17.wav
Analyzing file 11 of 500: /content/recordings/lucas/0_lucas_18.wav
Analyzing file 12 of 500: /content/recordings/lucas/0_lucas_19.wav
Analyzing file 13 of 500: /content/recordings/lucas/0_lucas_2.wav
Analyzing file 14 of 500: /content/recordings/lucas/0_lucas_20.wav
Analyzing file 15 of 500: /content/recordings/lucas/0_lucas_21.wav
Analyzi

(True, 'OK')